In [25]:
import librosa
import pandas as pd
import numpy as np

In [106]:
filename = '../Data/Alap/CC_8b_Bag/CC_8b_Bag-pitch.csv'
pitch_df = pd.read_csv(filename)

In [107]:
pitch_df

,time,pitch,energy
0,0.00,-3000.0,0.0
1,0.01,-3000.0,0.0
2,0.02,-3000.0,0.0
3,0.03,-3000.0,0.0
4,0.04,-3000.0,0.0
...,...,...,...
19737,197.37,-3000.0,0.0
19738,197.38,-3000.0,0.0
19739,197.39,-3000.0,0.0
19740,197.40,-3000.0,0.0


In [108]:
pitch_len = pitch_df.iloc[(np.diff(pitch_df['pitch'].values, prepend=np.nan) != 0).nonzero()][['time', 'pitch']].copy()

In [109]:
pitch_len['duration'] = np.diff(pitch_len['time'], append=(pitch_df.iloc[-1, 0]+0.1))

In [110]:
pitch_len['end time'] = pitch_len['time'] + pitch_len['duration']

In [111]:
pitch_len.loc[pitch_len['pitch'] == -3000]

,time,pitch,duration,end time
0,0.00,-3000.0,2.67,2.67
552,5.52,-3000.0,0.72,6.24
929,9.29,-3000.0,0.32,9.61
1087,10.87,-3000.0,2.07,12.94
1341,13.41,-3000.0,1.50,14.91
...,...,...,...,...
17751,177.51,-3000.0,0.44,177.95
18168,181.68,-3000.0,0.48,182.16
18323,183.23,-3000.0,0.01,183.24
18735,187.35,-3000.0,0.41,187.76


In [112]:
threshold = 0.5

In [113]:
bps = pitch_len.loc[(pitch_len['pitch'] == -3000) & (pitch_len['duration'] >= threshold)].reset_index(drop=True)

In [114]:
new_df_vals = {'start time': [], 'end time': [], 'duration': [], 'type': []}

In [115]:
for i, row in list(bps.iterrows())[:-1]:
    
    new_df_vals['start time'].append(row['time'])
    new_df_vals['end time'].append(row['end time'])
    new_df_vals['duration'].append(row['duration'])
    new_df_vals['type'].append('BP')
    
    new_df_vals['start time'].append(row['end time'])
    new_df_vals['end time'].append(bps.iloc[i+1, 0])
    new_df_vals['duration'].append(new_df_vals['end time'][-1] - new_df_vals['start time'][-1])
    new_df_vals['type'].append('SP')

In [116]:
phrases_df = pd.DataFrame(new_df_vals)
#phrases_df.to_csv('/home/nithya/Desktop/phrases2.csv', index=False)
sphrases_df = phrases_df.loc[phrases_df['type'] == 'SP'].reset_index()

In [117]:
seqs_dict = {'start time': [], 'end time': [], 'duration': [], 'singing duration': []}
for i, row in phrases_df.iterrows():
    if row['type'] == 'SP':
        if row['duration'] > 20:
            # the singing phrase is > 20s
            seqs_dict['start time'].append(row['start time'])
            seqs_dict['end time'].append(row['start time']+20)
        else:
            subset = phrases_df.loc[(phrases_df['start time'] >= row['start time']) & (phrases_df['end time'] <= row['start time']+20)]
            seqs_dict['start time'].append(row['start time'])
            seqs_dict['end time'].append(subset.iloc[-1, 1])
        seqs_dict['duration'].append(seqs_dict['end time'][-1] - seqs_dict['start time'][-1])
        seqs_dict['singing duration'].append(pitch_len.loc[(pitch_len['time'] >= seqs_dict['start time'][-1]) & (pitch_len['time'] <= seqs_dict['end time'][-1] ) & (pitch_len['pitch'] != -3000)].shape[0] / 100)

In [118]:
seqs = pd.DataFrame(seqs_dict)

In [119]:
#seqs.to_csv('/home/nithya/Desktop/seqs2.csv', index=False)

In [120]:
bps

,time,pitch,duration,end time
0,0.00,-3000.0,2.67,2.67
1,5.52,-3000.0,0.72,6.24
2,10.87,-3000.0,2.07,12.94
3,13.41,-3000.0,1.50,14.91
4,19.78,-3000.0,0.75,20.53
5,22.51,-3000.0,0.67,23.18
6,28.64,-3000.0,0.77,29.41
7,31.22,-3000.0,0.60,31.82
8,33.65,-3000.0,0.67,34.32
9,34.53,-3000.0,0.77,35.30


In [121]:
phrases_df

,start time,end time,duration,type
0,0.00,2.67,2.67,BP
1,2.67,5.52,2.85,SP
2,5.52,6.24,0.72,BP
3,6.24,10.87,4.63,SP
4,10.87,12.94,2.07,BP
...,...,...,...,...
67,163.48,167.73,4.25,SP
68,167.73,168.28,0.55,BP
69,168.28,170.66,2.38,SP
70,170.66,171.24,0.58,BP


In [122]:
seqs

,start time,end time,duration,singing duration
0,2.67,22.51,19.84,13.60
1,6.24,23.18,16.94,11.02
2,12.94,31.82,18.88,13.51
3,14.91,34.53,19.62,14.81
4,20.53,39.72,19.19,12.69
5,23.18,40.69,17.51,10.83
6,29.41,45.77,16.36,10.18
7,31.82,51.15,19.33,8.57
8,34.32,53.73,19.41,8.85
9,35.30,53.73,18.43,8.77


In [123]:
print('Number of BPs: ', str(phrases_df.loc[phrases_df['type'] == 'BP'].shape[0]))

Number of BPs:  36


In [124]:
print('Average duration of Singing Phrases: ', str(phrases_df.loc[phrases_df['type'] == 'SP', 'duration'].mean()))

Average duration of Singing Phrases:  4.305277777777777


In [125]:
print('Average duration of sequences sampled: ', str(seqs['duration'].mean()))

Average duration of sequences sampled:  17.13888888888889
